**1. What are the main motivations for reducing a dataset's dimensionality? What are the main drawbacks?**

"Apart from speeding up training, dimensionality reduction is also extremely useful for data visualization." As humans that live in a universe composed of 3 spatial dimensions, we have a lot of difficulty visualizing data that goes beyond the 3 dimensions. For example, something like an MNIST image will have 28\*28=784 dimensions which can't be plotted on a graph with just two or three axes. Reducing dimensionality will both speed up the training process and also make it possible (in most cases) to plot your data and make it readable to humans.

As for drawbacks, it's impossible to reduce a dataset's dimensionality without some loss, and thus will "make your system perform slightly worse." The book also makes the case that "it also makes your pipelines a bit more complex and thus harder to maintain," and while I think it's a valid point, I can't imagine that being anything beyond fairly trivial work.

**2. What is the curse of dimensionality? **
The curse of dimensionality is the accumulation of all the issues that arise from dealing with high-dimensional data. The inability to visualize or conceptualize, but also the fact that "high-dimensional datasets are at risk of being very sparse." This means that two instances of data are much more likely to be far away from eachother than in a low-dimensional dataset. This will increase the likelihood of overfitting.

**3. Once a dataset's dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?**

Theoretically, yes. In the kPCA algorithm example, we used the inverse transform function to reconstruct the dataset and evaluate the loss function compared to the original dataset. However, there will be a lot of loss, and the reconstruction will be nowhere near perfect.

Actually, ANN autoencoders can do this much better, because the whole point is to backpropogate to get the network to emulate the identity function as best as it can.

**4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?**

Regular PCA is going to work linearly, so no. However, Kernel PCA works really well for high-dimensional nonlinear data.

**5. Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?**

Well, I think there is no way to give an exact answer because every dataset will be different, but in most cases, you can reduce the dataset with PCA by 80% or so with an explained variance ratio of 95%, so approximately 200 dimensions.

**6. In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA, or Kernel PCA?**

Vanilla PCA is a good jumping off point. Unless you have a very complex dataset that folds over in many dimensions, it should work fine. 

However, if your dataset is larger than your memory, you can use Incremental PCA, which will take batches of data and partially fit the principal components. 

Randomized PCA has sort of the same idea, but this algorithm will quickly approximate the first *d* principal components, and will have a much smaller computational complexity than Incremental PCA when *d* is much smaller than *n*. So if you wanted to turn a 10,000-dimensional dataset into a 2-dimensional dataset (for whatever reason), you would run a lot faster using Randomized PCA.

Kernel PCA will work a lot better for doing complex nonlinear projections, and in many cases can unfold Manifolds.

**7. How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?**

Perhaps the easiest way is to reduce your dataset (or a subset of your dataset), and then run that reduced set through the `inverse_transform` function to return the dataset back to its original dimensionality. Then, you can use mean squared error or mean absolute error or whatever loss function you want to evaluate how much value was lost.

**8. Does it make any sense to chain two different dimensionality reduction algorithms?**

I suppose you could, maybe for example use LLE to unfold a twisted Manifold will result in the Manifold being unwrapped but still existing in 3 dimensional space on a plane not parallel to any of the standard planes. Then PCA would be good for defining two principle components to find the ideal plane to project the instances onto. Would this be better than just using one LLE algorithm to go from n dimensions to 2 dimensions? That would have to be tested on your dataset.

**9. Train a Random Forest MNIST classifier without dimensionality reduction and again with PCA. How does it perform differently?**

In [17]:
from sklearn.ensemble import RandomForestClassifier
from keras.datasets import mnist

In [18]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [19]:
import time

In [20]:
forest_clf = RandomForestClassifier(n_estimators=200, n_jobs=-1, max_depth=8)
time_start = time.time()
forest_clf.fit(x_train.reshape(-1,28*28), y_train)
time.time() - time_start

21.171406269073486

In [21]:
from sklearn.metrics import accuracy_score
predictions = forest_clf.predict(x_test.reshape(-1,28*28))
accuracy_score(y_test,predictions)

0.9298

The vanilla forest_clf with 784 input dimensions took **21 seconds** to train and got an accuracy score of **93%**.

In [25]:
from sklearn.decomposition import PCA

pca = PCA(n_components=.95)
x_reduced = pca.fit_transform(x_train.reshape(-1,28*28))
x_test_reduced = pca.transform(x_test.reshape(-1,28*28)) # Make sure you do transform and not fit_transform!
pca.n_components_

154

In [23]:
time_start = time.time()
forest_clf.fit(x_reduced, y_train)
time.time() - time_start

41.91395688056946

In [24]:
predictions = forest_clf.predict(x_test_reduced)
accuracy_score(y_test,predictions)

0.898

The forest_clf with PCA dimensionality reduction took **42 seconds** to train and got accuracy score of **90%**.